In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from database.BuildingRepository import BuildingRepository
from datetime import datetime, timedelta

In [3]:
data_set = pd.read_excel('./base_dados_102.xlsx')
data_set.head()

,Outside temperature (x10)(ºC),Temperature 102 (x10)(ºC),Heat Index 102 (x10)(ºC),Light 102 (%),AC status 102 (bool)
0,132,190,173,0,0
1,156,221,208,45,-1
2,156,221,208,45,-1
3,141,219,206,0,0
4,242,278,264,37,1


In [4]:
X = data_set[
    ['Outside temperature (x10)(ºC)', 'Temperature 102 (x10)(ºC)', 'Heat Index 102 (x10)(ºC)', 'Light 102 (%)']]
y = data_set['AC status 102 (bool)']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
clf = RandomForestClassifier(n_estimators=30, random_state=42)

In [7]:
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=30, random_state=42)

In [8]:
y_pred = clf.predict(X_test)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

Acurácia do modelo: 99.87%


In [10]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

Matriz de Confusão:
[[18553     0     0]
 [    0 20300    80]
 [    0     0 21110]]


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Precisão do modelo
precision = precision_score(y_test, y_pred, average='weighted')

# Recall
recall = recall_score(y_test, y_pred, average='weighted')

# F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

# resultados
print(f'Precisão: {precision:.10f}')
print(f'Recall: {recall:.10f}')
print(f'F1-score: {f1:.10f}')

Precisão: 0.9986726518
Recall: 0.9986676215
F1-score: 0.9986675714


In [12]:

from sklearn.model_selection import cross_val_score

# Validação cruzada (por exemplo, k-fold com k=5)
# Dividir o modelo em 5 partes e testa-lo
scores = cross_val_score(clf, X, y, cv=5)

# Pontuações obtidas
print("Pontuações da validação cruzada:", scores)

# Média das pontuações
print(f'Média das pontuações: {scores.mean():.10f}')

# Desvio padrão das pontuações
print(f'Desvio padrão das pontuações: {scores.std():.10f}')

Pontuações da validação cruzada: [0.99865097 0.9986676  0.9986676  0.99860098 0.99870091]
Média das pontuações: 0.9986576108
Desvio padrão das pontuações: 0.0000326361


In [13]:
import pickle

# Save the model to a file
model_filename = './modules/ACStatus/random_forest_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(clf, model_file)

In [14]:
import tkinter as tk
from sklearn.ensemble import RandomForestClassifier
# Previsões com base nos valores de entrada
def predict_ac_status():
    outside_temp = float(entry_outside_temp.get())
    temp_102 = float(entry_temp_102.get())
    heat_index_102 = float(entry_heat_index_102.get())
    light_102 = float(entry_light_102.get())

    # Previsão com base nos valores inseridos
    prediction = clf.predict([[outside_temp, temp_102, heat_index_102, light_102]])

    # Controlar o resultado para ser "Ligar AC em quente", "Ligar AC em frio" ou "desligar o AC"
    if prediction[0] == -1:
        result_label.config(text="Ligar AC em quente")
    elif prediction[0] == 1:
        result_label.config(text="Ligar AC em frio")
    else:
        result_label.config(text="Desligar o AC")


# Interface de usuário
root = tk.Tk()
root.title("Sistema de Controlo do AC")

frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

# Campos de entrada para as variáveis
entry_outside_temp = tk.Entry(frame, width=10, justify='center')
entry_temp_102 = tk.Entry(frame, width=10, justify='center')
entry_heat_index_102 = tk.Entry(frame, width=10, justify='center')
entry_light_102 = tk.Entry(frame, width=10, justify='center')

entry_outside_temp.grid(row=0, column=1)
entry_temp_102.grid(row=1, column=1)
entry_heat_index_102.grid(row=2, column=1)
entry_light_102.grid(row=3, column=1)

# Titulo das variáveis
tk.Label(frame, text="Outside Temperature (x10)(ºC)").grid(row=0, column=0)
tk.Label(frame, text="Temperature 102 (x10)(ºC)").grid(row=1, column=0)
tk.Label(frame, text="Heat Index 102 (x10)(ºC)").grid(row=2, column=0)
tk.Label(frame, text="Light 102 %").grid(row=3, column=0)

# Botão 'Prever'
predict_button = tk.Button(frame, text="Prever", command=predict_ac_status)
predict_button.grid(row=4, columnspan=2)

# Resultado
result_label = tk.Label(frame, text="")
result_label.grid(row=5, columnspan=2)
root.mainloop()


In [69]:
# Load the model from the file
with open(model_filename, 'rb') as model_file:
    clf = pickle.load(model_file)

In [70]:
building_repo = BuildingRepository()

df = pd.DataFrame(building_repo.get_historic(datetime.now() - timedelta(days=1)))
df = df.drop("_id", axis=1)
df.head()

,iots,datetime
0,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2023-11-29 11:47:41.155
1,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2023-11-29 11:47:36.349
2,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2023-11-29 11:47:31.100
3,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2023-11-29 11:47:26.377
4,"[{'name': 'Air Conditioner 101', 'type': 'hvac...",2023-11-29 11:47:21.086


In [77]:
aux = pd.DataFrame()
for i in range(1000):
    new = pd.DataFrame()
    for iot in df.iloc[i]['iots']:
        if iot['name'] in ['Light Sensor 103', 'Temperature Sensor 103', 'Humidity Sensor 103']:
            for value in iot['values']:
                val = value['values']
                new[iot['name'] + '_' + value['type']] = [val]

    aux = pd.concat([aux, new])
aux

,Temperature Sensor 103_temperature,Humidity Sensor 103_humidity,Light Sensor 103_light
0,233,631,565
0,233,631,565
0,233,631,563
0,233,631,565
0,233,631,565
...,...,...,...
0,216,655,599
0,216,655,599
0,216,655,600
0,216,655,600


In [78]:
def calculate_heat_index_custom_celsius(temperature, relative_humidity):
    c1 = -8.785
    c2 = 1.611
    c3 = 2.339
    c4 = -0.146
    c5 = -0.01231
    c6 = -0.01642
    c7 = .002212
    c8 = .0007255
    c9 = 0.000003582

    HI = (
            c1
            + c2 * temperature
            + c3 * relative_humidity
            + c4 * temperature * relative_humidity
            + c5 * temperature * temperature
            + c6 * relative_humidity * relative_humidity
            + c7 * temperature * temperature * relative_humidity
            + c8 * temperature * relative_humidity * relative_humidity
            + c9 * temperature * temperature * relative_humidity * relative_humidity
    )
    return HI

In [79]:
aux.rename(columns={'Temperature Sensor 103_temperature': 'Temperature (Cº)',
                    'Humidity Sensor 103_humidity': 'Humidity (%)', 'Light Sensor 103_light': 'Light (%)'},
           inplace=True)

aux['Temperature (Cº)'] = aux['Temperature (Cº)'] / 10
aux['Humidity (%)'] = aux['Humidity (%)'] / 10
aux['Light (%)'] = aux['Light (%)'] / 10
aux['Heat Index (ºC)'] = calculate_heat_index_custom_celsius(aux['Temperature (Cº)'], aux['Humidity (%)'])
aux['Outside temperature (ºC)'] = aux['Temperature (Cº)'] - 4

aux

,Temperature (Cº),Humidity (%),Light (%),Heat Index (ºC),Outside temperature (ºC)
0,23.3,63.1,56.5,40.451173,19.3
0,23.3,63.1,56.5,40.451173,19.3
0,23.3,63.1,56.3,40.451173,19.3
0,23.3,63.1,56.5,40.451173,19.3
0,23.3,63.1,56.5,40.451173,19.3
...,...,...,...,...,...
0,21.6,65.5,59.9,38.466646,17.6
0,21.6,65.5,59.9,38.466646,17.6
0,21.6,65.5,60.0,38.466646,17.6
0,21.6,65.5,60.0,38.466646,17.6


In [80]:
# Previsões com base nos valores de entrada
def predict_cenas(outside_temp, temperature, heat_index, light):
    # Previsão com base nos valores inseridos
    outside_temp = float(outside_temp)
    temperature = float(temperature)
    heat_index = float(heat_index)
    light = float(light)
    
    prediction = clf.predict([[outside_temp, temperature, heat_index, light]])

    # Controlar o resultado para ser "Ligar AC em quente", "Ligar AC em frio" ou "desligar o AC"
    # if prediction[0] == -1:
    #     print("Ligar AC em quente")
    # elif prediction[0] == 1:
    #     print("Ligar AC em frio")
    # else:
    #     print("Desligar o AC")
    
    return prediction[0]


In [81]:
predictions = []
for index, row in aux.iterrows():
    predictions.append(predict_cenas(row['Outside temperature (ºC)'], row['Temperature (Cº)'], row['Heat Index (ºC)'], row['Light (%)']))
aux['AC Status'] = predictions


/home/pardalito/miniconda3/envs/tiocps/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/pardalito/miniconda3/envs/tiocps/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/pardalito/miniconda3/envs/tiocps/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/pardalito/miniconda3/envs/tiocps/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/pardalito/miniconda3/envs/tiocps/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but Rando

In [84]:
aux

,Temperature (Cº),Humidity (%),Light (%),Heat Index (ºC),Outside temperature (ºC),AC Status
0,23.3,63.1,56.5,40.451173,19.3,0
0,23.3,63.1,56.5,40.451173,19.3,0
0,23.3,63.1,56.3,40.451173,19.3,0
0,23.3,63.1,56.5,40.451173,19.3,0
0,23.3,63.1,56.5,40.451173,19.3,0
...,...,...,...,...,...,...
0,21.6,65.5,59.9,38.466646,17.6,0
0,21.6,65.5,59.9,38.466646,17.6,0
0,21.6,65.5,60.0,38.466646,17.6,0
0,21.6,65.5,60.0,38.466646,17.6,0
